In [ ]:
import polars as pl
import datetime as dt
import numpy as np

from finance_byu.regtables import Regtable
import statsmodels.formula.api as smf

from factor_momentum._loaders import _scan_monthly_factor_returns

In [ ]:
from dotenv import load_dotenv
import os 

load_dotenv()

tmp = os.getenv('TMP')

deciles = pl.read_parquet(f'{tmp}/deciles.parquet')

In [ ]:
f_daily = pl.read_parquet(f'{tmp}/fama_french_factors.parquet').rename({"mkt_minus_rf": "mrk_excess"})

f_monthly = (f_daily.with_columns(
    pl.col('date').dt.truncate("1mo").alias('month')
)
.group_by(['month']).agg(
    [
        (np.log1p(pl.col(col).mul(.01)).sum())
        for col in f_daily.columns[1:]
    ]
)
.sort('month')
)

In [ ]:
merged = deciles.join(f_monthly, on="month", how="inner")
merged = merged.with_columns(
    pl.col('spread').sub(pl.col('rf')).alias('spread_excess')
)
merged.head()

In [ ]:
df = merged.to_pandas()

regformulas =  ['spread_excess ~ mrk_excess',
                'spread_excess ~ mrk_excess + smb + hml',
                'spread_excess ~ mrk_excess + smb + hml + rmw + cma']

reg = [smf.ols(f, df).fit() for f in regformulas]
tbl = Regtable(reg)
tbl.render()

In [ ]:
np.exp(0.08) - 1